In [1]:
import aiofiles
import aiohttp
import asyncio
import csv

async def retrieve_pingdom_checks(api_key):
    url = 'https://api.pingdom.com/api/3.1/checks'
    headers = {
        'Authorization': f'Bearer {api_key}'
    }

    try:
        async with aiohttp.ClientSession(headers=headers) as session:
            async with session.get(url) as response:
                response.raise_for_status()
                result = await response.json()
                return result.get('checks', [])
    except aiohttp.ClientError as e:
        print(f"Failed to retrieve Pingdom checks: {e}")
        return []

async def perform_pingdom_check(api_key, check_id, target_url, csv_writer):
    url = f'https://api.pingdom.com/api/3.1/single?type=http&host={target_url}'
    headers = {
        'Authorization': f'Bearer {api_key}'
    }

    try:
        async with aiohttp.ClientSession(headers=headers) as session:
            async with session.get(url) as response:
                response.raise_for_status()
                result = await response.json()
                result_data = result.get('result', None)
                if result_data:
                    status = result_data.get('status', 'N/A')
                    probedesc = result_data.get('probedesc', 'N/A')
                    statusdesc = result_data.get('statusdesc', 'N/A')
                    statusdesclong = result_data.get('statusdesclong', 'N/A')
                    await csv_writer.writerow([target_url, status, probedesc, statusdesc, statusdesclong, check_id])
                else:
                    await csv_writer.writerow([target_url, "No result returned", check_id])
    except aiohttp.ClientError as e:
        await csv_writer.writerow([target_url, f"Error: {e}", check_id])

async def main(api_key, target_urls):
    checks = await retrieve_pingdom_checks(api_key)
    if not checks:
        print("No Pingdom checks found.")
        return

    async with aiofiles.open('pingdom_results.csv', mode='w', newline='') as csv_file:
        csv_writer = csv.writer(csv_file)
        await csv_writer.writerow(['Target URL', 'Status', 'Probe Description', 'Status Description', 'Long Status Description', 'Pingdom Check ID'])
        
        tasks = []
        for check in checks:
            check_id = check.get('id')
            hostname = check.get('hostname')
            if hostname in target_urls:
                task = perform_pingdom_check(api_key, check_id, hostname, csv_writer)
                tasks.append(task)

        await asyncio.gather(*tasks)

# Example usage
api_key = 'zEbEAu_VNxQWBkenvLetuQrwhYgm_F0srrpcRLjjAZ-3aKMGO5nK1pRn5vNWeylI6TupRp4'
target_urls_input = input("Enter the target URLs separated by commas: ")
target_urls = [url.strip() for url in target_urls_input.split(",")]

await main(api_key, target_urls)
